https://huggingface.co/docs/trl/v0.4.2/sft_trainer  
위 공식 문서를 필독하시기를 권합니다.

해당 실습은 Colab Pro 버전에서 A100 GPU 선택 후 진행되었습니다.

## 1. 셋팅

In [17]:
!wget https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/23.%20LLM%20Fine-tuning/sft-trainer-alpaca.py
!wget https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/23.%20LLM%20Fine-tuning/requirements.txt

--2024-02-17 14:33:00--  https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/23.%20LLM%20Fine-tuning/sft-trainer-alpaca.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12502 (12K) [text/plain]
Saving to: ‘sft-trainer-alpaca.py’

sft-trainer-alpaca. 100%[===================>]  12.21K  --.-KB/s    in 0s      

2024-02-17 14:33:00 (108 MB/s) - ‘sft-trainer-alpaca.py’ saved [12502/12502]

--2024-02-17 14:33:00--  https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/23.%20LLM%20Fine-tuning/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... c

In [18]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.9/257.9 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.6 MB/s eta 0:00:00


## 2. LLM 학습 데이터 유형

In [6]:
from datasets import load_dataset

# dataset testing
dataset = load_dataset("kyujinpy/KOpen-platypus")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/24926 [00:00<?, ? examples/s]

In [7]:
print(dataset['train']['instruction'][0])

보드 게임 스피너는 $A$, $B$, $C$로 표시된 세 부분으로 나뉩니다. 스피너가 $A$에 떨어질 확률은 $\frac{1}{3}$이고, 스피너가 $B$에 떨어질 확률은 $\frac{5}{12}$입니다.  스피너가 $C$에 착륙할 확률은 얼마입니까? 답을 공통 분수로 표현하세요.


In [8]:
print(dataset['train']['output'][0])

모든 가능한 결과의 확률의 합이 1$이므로, 스피너가 $C$에 착륙할 확률을 구하려면 스피너가 $A$와 $B$에 착륙할 확률을 1$에서 빼야 합니다. 이를 방정식으로 쓸 수 있습니다: $P(C) = 1 - P(A) - P(B)$. P(A) = \frac{1}{3}$, $P(B) = \frac{5}{12}$라는 것을 알고 있으므로 이 값을 방정식에 대입하여 단순화할 수 있습니다. 결과는 다음과 같습니다: P(C) = 1 - \frac{1}{3} - frac{5}{12} = \frac{12}{12} - frac{4}{12} - frac{5}{12} = \frac{3}{12}$. 분자와 분모를 $3$로 나누면 이 분수를 줄일 수 있습니다: P(C) = \frac{1}{4}$입니다.


In [9]:
print(dataset['train']['instruction'][1])

저희 학교 수학 클럽에는 남학생 6명과 여학생 8명이 있습니다.  주 수학 경시대회에 파견할 팀을 선발해야 합니다. 팀에 6명이 필요합니다.  제한 없이 팀을 몇 가지 방법으로 선발할 수 있나요?


In [10]:
print(dataset['train']['output'][1])

14명 중 6명을 선택해야 하는데 순서는 중요하지 않습니다. 이것은 순열 문제가 아니라 조합 문제입니다. 조합의 공식은 nCr = n! / (r! * (n-r)!)이며, 여기서 n은 총 선택의 개수이고 r은 선택의 개수입니다. 숫자를 연결하면 14C6 = 14! / (6! * 8!) = 3003.


## 3. 튜닝

싱글 GPU를 사용하고 있다면 sft-trainer-alpaca.py에서 ddp_find_unused_paramters=False를 사용하세요.

if you are using 1 GPU, please change ddp_find_unused_paramters=False in the HF trainer. We will be updating the fine-tuning script to handle these changes automatically.

In [22]:
!torchrun sft-trainer-alpaca.py \
    --base_model beomi/llama-2-ko-7b \
    --data-path kyujinpy/KOpen-platypus \
    --output_dir ./fine_tuned_llama-2-ko-7b \
    --batch_size 64 \
    --micro_batch_size 1 \
    --num_epochs 15 \
    --learning_rate 1e-5 \
    --cutoff_len 2048 \
    --val_set_size 0 \
    --lora_r 16 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --lora_target_modules '[gate_proj, down_proj, up_proj]' \
    --train_on_inputs False \
    --add_eos_token True \
    --group_by_length False \
    --lr_scheduler 'linear' \
    --warmup_steps 0
#--resume_from_checkpoint '/fine_tuned_llama-2-ko-7b/checkpoint-400'

base_model: beomi/llama-2-ko-7b
data_path: kyujinpy/KOpen-platypus
output_dir: ./fine_tuned_llama-2-ko-7b
batch_size: 64
micro_batch_size: 1
num_epochs: 15
learning_rate: 1e-05
cutoff_len: 2048
val_set_size: 0
lr_scheduler: linear
warmup_steps: 0
lora_r: 16
lora_alpha: 16
lora_dropout: 0.05
lora_target_modules: ['gate_proj', 'down_proj', 'up_proj']
train_on_inputs: False
add_eos_token: True
group_by_length: False
resume_from_checkpoint: False

gpu 개수: 1
Loading checkpoint shards: 100% 15/15 [00:08<00:00,  1.72it/s]
pre-trained model's BOS EOS and PAD token id: 1 2 2
/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:141: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
trainable params: 23,199,744 || all params: 6,879,055,872 || trainable%: 0.3372518617624625
Map: 100% 24926/24926 [00:43<00:00, 571.00 examples/s]
### 첫번째 샘플 출력 ###
{'input': '솔루션으로 A, B, C 또는 D를 선택

## 4. Merging

In [23]:
!wget https://raw.githubusercontent.com/ukairia777/LLM-Finetuning-tutorial/main/merge.py

--2024-02-17 15:11:18--  https://raw.githubusercontent.com/ukairia777/LLM-Finetuning-tutorial/main/merge.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1351 (1.3K) [text/plain]
Saving to: ‘merge.py’

merge.py            100%[===================>]   1.32K  --.-KB/s    in 0s      

2024-02-17 15:11:18 (93.2 MB/s) - ‘merge.py’ saved [1351/1351]



In [26]:
%ls -al

total 48
drwxr-xr-x 1 root root  4096 Feb 17 15:11 ./
drwxr-xr-x 1 root root  4096 Feb 17 14:22 ../
drwxr-xr-x 4 root root  4096 Feb 14 14:27 .config/
drwxr-xr-x 3 root root  4096 Feb 17 14:36 fine_tuned_llama-2-ko-7b/
drwxr-xr-x 2 root root  4096 Feb 17 14:28 .ipynb_checkpoints/
-rw-r--r-- 1 root root  1351 Feb 17 15:11 merge.py
-rw-r--r-- 1 root root   154 Feb 17 14:33 requirements.txt
drwxr-xr-x 1 root root  4096 Feb 14 14:28 sample_data/
-rw-r--r-- 1 root root 12451 Feb 17 14:43 sft-trainer-alpaca.py


In [ ]:
!python merge.py \
    --base_model_name_or_path beomi/llama-2-ko-7b \
    --peft_model_path ./fine_tuned_llama-2-ko-7b/checkpoint-1000 \
    --output_dir ./fine_tuned_llama-2-ko-7b/merged